In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

dummy_reg = DummyRegressor(strategy="mean")

Test_Target = ["SUSPECT_ARRESTED_FLAG", "STOP_WAS_INITIATED",
                 "OBSERVED_DURATION_MINUTES", "SUSPECTED_CRIME_DESCRIPTION",
                 "STOP_DURATION_MINUTES", "FIREARM_FLAG",
                 "SUSPECT_RACE_DESCRIPTION", "SUSPECT_HEIGHT",
                 "DEMEANOR_OF_PERSON_STOPPED", "SUSPECT_BODY_BUILD_TYPE"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)
Feature_test=["FIREARM_FLAG", "STOP_DURATION_MINUTES", "STOP_DURATION_MINUTES"]
#C_Feature_test=["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION","SUSPECT_RACE_DESCRIPTION","DEMEANOR_OF_PERSON_STOPPED"]



Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', float('nan'), inplace=True)

lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])
X = Test1[["SUSPECT_ARRESTED_FLAG"]]
y = Test1[Feature_test]
Test1.to_csv('output.csv', index=False)

Label_Encode_feature = ["STOP_WAS_INITIATED"]
Label_En = OneHotEncoder(handle_unknown="ignore")
Feature_Encoded = Label_En.fit_transform(Test1[Label_Encode_feature])
Label_df = pd.DataFrame(Feature_Encoded.toarray(), columns=Label_En.get_feature_names_out(Label_Encode_feature))
Label_df.fillna(value=0, inplace=True)



Test1 = pd.concat([Test1, Label_df], axis=1)

# drop the original categorical feature column
Test1 = Test1.drop(columns=Label_Encode_feature)



train_data,test_data=train_test_split(Test1,test_size=0.2,random_state=42)

#dummy_reg.fit(X_train, y_train)
#y_train_predict = dummy_reg.predict(X_train)

#trainerror = mean_squared_error(y_train, y_train_predict, squared=False)

#y_test_pred = dummy_reg.predict(X_test)
#testerror = mean_squared_error(y_test, y_test_pred, squared=False)

#print("Training error:", trainerror)
#print("Test error:", testerror)



In [28]:



model_R = RandomForestClassifier(criterion='entropy')
#model_R.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])
model_R.fit(Label_df, train_data["SUSPECT_ARRESTED_FLAG"])
y_pred=model_R.predict(train_data[Label_df])
accuracy = accuracy_score(train_data["SUSPECT_ARRESTED_FLAG"], y_pred)
print('Accuracy:', accuracy)


ValueError: Found input variables with inconsistent numbers of samples: [15102, 9925]

In [ ]:
model = LinearRegression()
model.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])
predictions=model.predict(test_data[Feature_test])
train_predictions=model.predict(train_data[Feature_test])
rmse = mean_squared_error(test_data["SUSPECT_ARRESTED_FLAG"],predictions,squared=False)
print('RMSE: ', rmse)
train_rmse=mean_squared_error(train_data["SUSPECT_ARRESTED_FLAG"],train_predictions,squared=False)
print("Train_RMSE:",train_rmse)

RMSE:  0.43523548445268373
Train_RMSE: 0.4394223770188287
